# deeplay.vision

Does not exist yet.

# deeplay.language

Does not exist yet.

# deeplay.geometric

Does not exist yet.

# deeplay.activelearning

Contains tools for active learning.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cluster1 = np.random.normal(0, 1, (1000, 2)) - (2, 0)
cluster2 = np.random.normal(0, 1, (1000, 2)) + (2, 0)

test_cluster1 = np.random.normal(0, 1, (100, 2)) - (2, 0)
test_cluster2 = np.random.normal(0, 1, (100, 2)) + (2, 0)

plt.scatter(cluster1[:, 0], cluster1[:, 1], s=2)
plt.scatter(cluster2[:, 0], cluster2[:, 1], s=2)



In [ ]:
net = dl.MultiLayerPerceptron(2, [10], 2)
classifier = dl.BinaryClassifier(net)

In [ ]:
import deeplay.activelearning as al
import torch
# Creating an active learning dataset

torch_dataset = torch.utils.data.TensorDataset(
    torch.tensor(np.vstack([cluster1, cluster2]), dtype=torch.float32),
    torch.tensor([0] * 1000 + [1] * 1000, dtype=torch.long)
)

test_torch_dataset = torch.utils.data.TensorDataset(
    torch.tensor(np.vstack([test_cluster1, test_cluster2]), dtype=torch.float32),
    torch.tensor([0] * 100 + [1] * 100, dtype=torch.long)
)

train_dataset = al.ActiveLearningDataset(torch_dataset)
# Initialize the dataset
train_dataset.annotate_random(4)

# Creating an active learning model
al_model = al.UncertaintyStrategy(classifier, criterion=al.Margin(), train_pool=train_dataset, batch_size=8).build()
print(al_model)

In [ ]:
import torchmetrics as tm
accuracy_score = []
num_labeled = []

for iter in range(10):
    trainer = dl.Trainer(max_epochs=20)
    trainer.fit(al_model)
    
    res = al_model.test(test_torch_dataset, {"accuracy": tm.Accuracy("multiclass", num_classes=2)})

    X, y = train_dataset.get_annotated_samples(), train_dataset.get_annotated_labels()

    plt.scatter(cluster1[:, 0], cluster1[:, 1], s=1, c="gray")
    plt.scatter(cluster2[:, 0], cluster2[:, 1], s=1, c="gray")
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c="C0")
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c="C1")
    plt.show()

    accuracy_score.append(res["accuracy"].cpu())
    num_labeled.append(train_dataset.get_num_annotated())

    # Update the dataset
    al_model.query_and_update(4)
    al_model.reset_model()

plt.plot(num_labeled, accuracy_score)